# begin

In [ ]:
batch_size = 128
epochs = 20
def one_hot_encode(y):
    l = list()
    for item in y:
        c = [0. for i in range(10)]
        c[item] = 1.
        l.append(c)
    return np.array(l)
def make_model(input_shape):
    input_tensor = Input(input_shape)
    x = input_tensor
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)  # 增加一个隐藏层
    x = Dense(10, activation='softmax')(x)
    model = Model(input_tensor, x)
    return model
X_train = []
X_valid = []

premodels = [
    "bottleneck_ResNet50.h5",
    "bottleneck_Xception.h5",
    "bottleneck_InceptionV3.h5",
]

for filename in premodels:
    print('------------------'+filename)
    with h5py.File(os.path.join("models", tag, filename), 'r') as h:
        X_train.append(np.array(h['train']))
        X_valid.append(np.array(h['valid']))
        y_train = np.array(h['label'])
        y_valid = np.array(h['valid_label'])

X_train = np.concatenate(X_train, axis=1)
X_valid = np.concatenate(X_valid, axis=1)

X_train, y_train = shuffle(X_train, y_train)
y_train = one_hot_encode(y_train)
X_valid, y_valid = shuffle(X_valid, y_valid)
y_valid = one_hot_encode(y_valid)
model_mix = make_model(X_train.shape[1:])

tensorboard_callback = keras.callbacks.TensorBoard()

print("Adam")
model_mix.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model_mix.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_valid,y_valid), callbacks=[tensorboard_callback])
model_mix.save("models/mixed-model.h5")

In [34]:
import numpy as np
import os
import h5py
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

# 设置超参数
batch_size = 128
epochs = 20

# 数据处理函数：使用Keras的to_categorical来代替one_hot_encode
def one_hot_encode(y):
    return to_categorical(y, num_classes=10)

# 创建混合模型
def create_model(input_shape):
    # 输入层
    input_tensor = Input(input_shape)

    # 增加Dropout层和隐藏层
    x = Dropout(0.5)(input_tensor)  # 对输入数据进行Dropout
    x = Dense(512, activation='relu')(x)
    
    # 输出层
    output = Dense(10, activation='softmax')(x)
    
    # 创建最终模型
    model = Model(inputs=input_tensor, outputs=output)
    return model

# 读取数据
X_train = []
X_valid = []
y_train = None
y_valid = None

premodels = [
    "bottleneck_ResNet50.h5",
    "bottleneck_Xception.h5",
    "bottleneck_InceptionV3.h5",
]

for filename in premodels:
    print('------------------' + filename)
    with h5py.File(os.path.join("models", tag, filename), 'r') as h:
        X_train.append(np.array(h['train']))
        X_valid.append(np.array(h['valid']))
        y_train = np.array(h['label'])
        y_valid = np.array(h['valid_label'])

# 拼接多个模型的特征
X_train = np.concatenate(X_train, axis=1)
X_valid = np.concatenate(X_valid, axis=1)

# Shuffle 数据
X_train, y_train = shuffle(X_train, y_train)
y_train = one_hot_encode(y_train)
X_valid, y_valid = shuffle(X_valid, y_valid)
y_valid = one_hot_encode(y_valid)

# 创建混合模型
model_mix = create_model(X_train.shape[1:])  # 直接使用已拼接的特征

# 使用TensorBoard回调
tensorboard_callback = TensorBoard(log_dir='./logs')

# 使用ModelCheckpoint回调来保存最佳模型
checkpoint_callback = ModelCheckpoint(
    'models/mixed-model-best.h5',
    save_best_only=True,
    monitor='val_loss',
    mode='min'
)

# 编译模型
model_mix.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
model_mix.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_valid, y_valid),
    callbacks=[tensorboard_callback, checkpoint_callback]
)

# 保存最终模型
model_mix.save("models/mixed-model.h5")


------------------bottleneck_ResNet50.h5
------------------bottleneck_Xception.h5
------------------bottleneck_InceptionV3.h5
Epoch 1/20
163/163 [==============================] - 5s 29ms/step - loss: 0.0682 - accuracy: 0.9846 - val_loss: 0.7252 - val_accuracy: 0.7575
Epoch 2/20
163/163 [==============================] - 5s 28ms/step - loss: 0.0026 - accuracy: 0.9994 - val_loss: 0.8314 - val_accuracy: 0.7495
Epoch 3/20
163/163 [==============================] - 5s 28ms/step - loss: 0.0019 - accuracy: 0.9995 - val_loss: 0.7875 - val_accuracy: 0.7618
Epoch 4/20
163/163 [==============================] - 5s 28ms/step - loss: 0.0018 - accuracy: 0.9994 - val_loss: 0.8098 - val_accuracy: 0.7740
Epoch 5/20
163/163 [==============================] - 5s 28ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.8514 - val_accuracy: 0.7599
Epoch 6/20
163/163 [==============================] - 5s 28ms/step - loss: 7.3260e-04 - accuracy: 0.9999 - val_loss: 1.2439 - val_accuracy: 0.7141
Epoch 7/20
1

In [36]:
import keras
print(keras.__version__)
import tensorflow as tf
print(tf.__version__)


2.12.0
2.12.0


# 00000000

In [1]:
import os
import h5py
import numpy as np
from sklearn.utils import shuffle
import pandas as pd

import keras
from keras.preprocessing.image import *
from keras.optimizers import *
from keras.callbacks import *

np.random.seed(2017)

the ipynb is refer: 
Create the premodel by ResNet50,Xception, InceptionV3, VGG16, VGG19

In [2]:
# dir = "/ext/Data/distracted_driver_detection/"
# dir = "F:\BBBBBBBBBBBBBBBBB\Datas"
dir = "F:\JupyterWorkSpace\BBBBBBS"
tag = "finetune"

def one_hot_encode(y):
    l = list()
    for item in y:
        c = [0. for i in range(10)]
        c[item] = 1.
        l.append(c)
    return np.array(l)
    
from keras.models import *
from keras.layers import *

# def make_model(input_shape):

#     input_tensor = Input(input_shape)
#     x = input_tensor
#     x = Dropout(0.5)(x)
#     x = Dense(10, activation='softmax')(x)
#     model = Model(input_tensor, x)
#     return model

def make_model(input_shape):
    input_tensor = Input(input_shape)
    x = input_tensor
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)  # 增加一个隐藏层
    x = Dense(10, activation='softmax')(x)
    model = Model(input_tensor, x)
    return model


print("Done")

Done


parameters

In [3]:
batch_size = 128
epochs = 20

# Combine multi-modles

In [4]:
import h5py
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
np.random.seed(2017)

X_train = []
X_valid = []

premodels = [
    "bottleneck_ResNet50.h5",
    "bottleneck_Xception.h5",
    "bottleneck_InceptionV3.h5",
]

for filename in premodels:
    print('------------------'+filename)
    with h5py.File(os.path.join("models", tag, filename), 'r') as h:
        X_train.append(np.array(h['train']))
        X_valid.append(np.array(h['valid']))
        y_train = np.array(h['label'])
        y_valid = np.array(h['valid_label'])

X_train = np.concatenate(X_train, axis=1)
X_valid = np.concatenate(X_valid, axis=1)

X_train, y_train = shuffle(X_train, y_train)
y_train = one_hot_encode(y_train)
X_valid, y_valid = shuffle(X_valid, y_valid)
y_valid = one_hot_encode(y_valid)

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)


------------------bottleneck_ResNet50.h5
------------------bottleneck_Xception.h5
------------------bottleneck_InceptionV3.h5
(20787, 6144)
(20787, 10)
(1637, 6144)
(1637, 10)


In [5]:
X_train.shape[1:]

(6144,)

In [6]:
model_mix = make_model(X_train.shape[1:])

tensorboard_callback = keras.callbacks.TensorBoard()

print("Adam")
model_mix.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model_mix.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_valid,y_valid), callbacks=[tensorboard_callback])
# print("RMSprop")
# model_mix.compile(optimizer=RMSprop(lr=1*0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
# model_mix.fit(X_train, y_train, batch_size=batch_size, epochs=20, validation_data=(X_valid,y_valid))

model_mix.save("models/mixed-model.h5")
print("model save successed")

Adam
Epoch 1/10


C:\Users\13904\anaconda3\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


163/163 [==============================] - 3s 16ms/step - loss: 0.5746 - accuracy: 0.8794 - val_loss: 0.7015 - val_accuracy: 0.7514
Epoch 2/10
163/163 [==============================] - 2s 15ms/step - loss: 0.0401 - accuracy: 0.9984 - val_loss: 0.6447 - val_accuracy: 0.7618
Epoch 3/10
163/163 [==============================] - 2s 15ms/step - loss: 0.0182 - accuracy: 0.9987 - val_loss: 0.6543 - val_accuracy: 0.7587
Epoch 4/10
163/163 [==============================] - 2s 15ms/step - loss: 0.0117 - accuracy: 0.9989 - val_loss: 0.6658 - val_accuracy: 0.7587
Epoch 5/10
163/163 [==============================] - 2s 15ms/step - loss: 0.0085 - accuracy: 0.9990 - val_loss: 0.6837 - val_accuracy: 0.7575
Epoch 6/10
163/163 [==============================] - 2s 15ms/step - loss: 0.0069 - accuracy: 0.9990 - val_loss: 0.6953 - val_accuracy: 0.7569
Epoch 7/10
163/163 [==============================] - 2s 15ms/step - loss: 0.0054 - accuracy: 0.9993 - val_loss: 0.7180 - val_accuracy: 0.7544
Epoch 8/10

In [7]:
model_mix

In [62]:
# def gen_kaggle_csv(model, X_test,  model_image_size, csv_name):
#     y_pred = model.predict(X_test, verbose=1)
#     print(y_pred[:3])
#     y_pred = y_pred.clip(min=0.005, max=0.995)
#     print()
#     print(y_pred[:3])

#     gen = ImageDataGenerator()
#     test_generator = gen.flow_from_directory(dir  + "/test/", (model_image_size, model_image_size), shuffle=False, 
#                                              batch_size=16, class_mode=None)

#     l = list()
#     for i, fname in enumerate(test_generator.filenames):
#         name = fname[fname.rfind('/')+1:]
#         l.append( [name, *y_pred[i]] )

#     l = np.array(l)
#     data = {'img': l[:,0]}
#     for i in range(10):
#         data["c%d"%i] = l[:,i+1]
#     df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
#     df.head(10)
#     df = df.sort_values(by='img')
#     df.to_csv(csv_name, index=None, float_format='%.3f')
    
    
#     print(f"Number of files: {len(test_generator.filenames)}")
#     print(f"Number of predictions: {len(y_pred)}")

# print("done")


done


In [8]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def gen_kaggle_csv(model, X_test, model_image_size, csv_name):
    # 对测试数据进行预测
    y_pred = model.predict(X_test, verbose=1)
    print(y_pred[:3])  # 打印前3个预测结果
    
    # 对预测结果进行裁剪，确保它们在合理范围内
    y_pred = y_pred.clip(min=0.005, max=0.995)
    print()
    print(y_pred[:3])  # 打印裁剪后的前3个预测结果

    # 创建图像数据生成器
    gen = ImageDataGenerator()

    # 使用 flow_from_directory 加载测试数据集，不打乱顺序，设置目标大小
    test_generator = gen.flow_from_directory(
        dir + "/test/",  # 测试集文件夹路径
        (model_image_size, model_image_size),  # 调整图像大小
        shuffle=False,  # 不打乱图像顺序
        batch_size=16,  # 设置每批处理的图像数量
        class_mode=None  # 因为没有标签，设置为 None
    )

    # 创建一个空列表来存储文件名和预测值
    l = list()

    # 遍历测试集的文件名，提取图片名称并与预测结果结合
    for i, fname in enumerate(test_generator.filenames):
        name = os.path.basename(fname)  # 获取文件名，不包含路径部分
        l.append([name, *y_pred[i]])

    # 将列表转换为 NumPy 数组
    l = np.array(l)

    # 创建字典，存储图片名称和预测概率
    data = {'img': l[:, 0]}  # 图片名称
    for i in range(10):
        data["c%d" % i] = l[:, i + 1]  # 每个类别的预测概率

    # 将字典转换为 DataFrame，列名是图片名称和类别预测概率
    df = pd.DataFrame(data, columns=['img'] + ['c%d' % i for i in range(10)])

    # 打印前10行数据检查
    print(df.head(10))

    # 按照图片名称排序
    df = df.sort_values(by='img')

    # 将 DataFrame 保存为 CSV 文件
    df.to_csv(csv_name, index=None, float_format='%.3f')

    # 打印测试集中图片数量和预测结果数量，进行简单的检查
    print(f"Number of files: {len(test_generator.filenames)}")
    print(f"Number of predictions: {len(y_pred)}")

# 打印完成信息
print("done")


done


In [9]:
test_premodels = [
     "bottleneck_ResNet50_test.h5", 
     "bottleneck_Xception_test.h5", 
     "bottleneck_InceptionV3_test.h5",
]
X_test = []
for filename in test_premodels:
    print('------------------'+filename)
    with h5py.File(os.path.join("models", tag, filename), 'r') as h:
        X_test.append(np.array(h['test']))
        
X_test = np.concatenate(X_test, axis=1)
gen_kaggle_csv(model_mix, X_test,  320, 'csv/mixed-pred-first.csv')

------------------bottleneck_ResNet50_test.h5
------------------bottleneck_Xception_test.h5
------------------bottleneck_InceptionV3_test.h5
2492/2492 [==============================] - 6s 2ms/step
[[0.01632781 0.01399498 0.02791261 0.02079366 0.04007988 0.73943186
  0.06469563 0.02096968 0.02348928 0.03230467]
 [0.04596654 0.01829088 0.01733022 0.01399086 0.05013722 0.68175805
  0.06910319 0.01904308 0.07859012 0.00578985]
 [0.16679934 0.12803547 0.02558471 0.04717511 0.08069336 0.03078406
  0.11685694 0.07124319 0.07661813 0.25620964]]

[[0.01632781 0.01399498 0.02791261 0.02079366 0.04007988 0.73943186
  0.06469563 0.02096968 0.02348928 0.03230467]
 [0.04596654 0.01829088 0.01733022 0.01399086 0.05013722 0.68175805
  0.06910319 0.01904308 0.07859012 0.00578985]
 [0.16679934 0.12803547 0.02558471 0.04717511 0.08069336 0.03078406
  0.11685694 0.07124319 0.07661813 0.25620964]]
Found 79726 images belonging to 1 classes.
              img           c0           c1            c2         

# 只关于输出结果部分的汇总（假定已经得到了bottleneck_ResNet50_test等）

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

import h5py
from sklearn.utils import shuffle

import keras
from keras.preprocessing.image import *
from keras.optimizers import *
from keras.callbacks import *

def gen_kaggle_csv(model, X_test, model_image_size, csv_name):
    # 对测试数据进行预测
    y_pred = model.predict(X_test, verbose=1)
    print(y_pred[:3])  # 打印前3个预测结果
    
    # 对预测结果进行裁剪，确保它们在合理范围内
    y_pred = y_pred.clip(min=0.005, max=0.995)
    print()
    print(y_pred[:3])  # 打印裁剪后的前3个预测结果

    # 创建图像数据生成器
    gen = ImageDataGenerator()

    # 使用 flow_from_directory 加载测试数据集，不打乱顺序，设置目标大小
    test_generator = gen.flow_from_directory(
        dir + "/test/",  # 测试集文件夹路径
        (model_image_size, model_image_size),  # 调整图像大小
        shuffle=False,  # 不打乱图像顺序
        batch_size=16,  # 设置每批处理的图像数量
        class_mode=None  # 因为没有标签，设置为 None
    )

    # 创建一个空列表来存储文件名和预测值
    l = list()

    # 遍历测试集的文件名，提取图片名称并与预测结果结合
    for i, fname in enumerate(test_generator.filenames):
        name = os.path.basename(fname)  # 获取文件名，不包含路径部分
        l.append([name, *y_pred[i]])

    # 将列表转换为 NumPy 数组
    l = np.array(l)

    # 创建字典，存储图片名称和预测概率
    data = {'img': l[:, 0]}  # 图片名称
    for i in range(10):
        data["c%d" % i] = l[:, i + 1]  # 每个类别的预测概率

    # 将字典转换为 DataFrame，列名是图片名称和类别预测概率
    df = pd.DataFrame(data, columns=['img'] + ['c%d' % i for i in range(10)])

    # 打印前10行数据检查
    print(df.head(10))

    # 按照图片名称排序
    df = df.sort_values(by='img')

    # 将 DataFrame 保存为 CSV 文件
    df.to_csv(csv_name, index=None, float_format='%.3f')

    # 打印测试集中图片数量和预测结果数量，进行简单的检查
    print(f"Number of files: {len(test_generator.filenames)}")
    print(f"Number of predictions: {len(y_pred)}")

# 打印完成信息
print("done")

from tensorflow.keras.models import load_model
# 加载已保存的模型
model_mix = load_model("models/mixed-model.h5")
# 现在你可以使用 model_mix 来进行预测或其他操作

dir = "F:\JupyterWorkSpace\BBBBBBS"

test_premodels = [
     "bottleneck_ResNet50_test.h5", 
     "bottleneck_Xception_test.h5", 
     "bottleneck_InceptionV3_test.h5",
]
X_test = []
for filename in test_premodels:
    print('------------------'+filename)
    with h5py.File(os.path.join("models", "finetune", filename), 'r') as h:
        X_test.append(np.array(h['test']))
        
X_test = np.concatenate(X_test, axis=1)
gen_kaggle_csv(model_mix, X_test,  320, 'csv/mixed-pred-test.csv')

done
------------------bottleneck_ResNet50_test.h5
------------------bottleneck_Xception_test.h5
------------------bottleneck_InceptionV3_test.h5
2492/2492 [==============================] - 6s 2ms/step
[[0.0063405  0.01604431 0.02785161 0.02301869 0.03530462 0.8023355
  0.02995177 0.01099865 0.01762458 0.03052977]
 [0.02174795 0.01262218 0.02817013 0.01803399 0.02463043 0.81699604
  0.01878481 0.00956798 0.04342525 0.00602125]
 [0.16492495 0.07910203 0.05841956 0.06612758 0.09217478 0.03954029
  0.07231129 0.11545973 0.11070835 0.20123146]]

[[0.0063405  0.01604431 0.02785161 0.02301869 0.03530462 0.8023355
  0.02995177 0.01099865 0.01762458 0.03052977]
 [0.02174795 0.01262218 0.02817013 0.01803399 0.02463043 0.81699604
  0.01878481 0.00956798 0.04342525 0.00602125]
 [0.16492495 0.07910203 0.05841956 0.06612758 0.09217478 0.03954029
  0.07231129 0.11545973 0.11070835 0.20123146]]
Found 79726 images belonging to 1 classes.
              img           c0           c1           c2       

# 规范代码

In [2]:
import os
import numpy as np
import pandas as pd
import h5py
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

def gen_kaggle_csv(model, X_test, model_image_size, csv_name):
    """
    Generate Kaggle submission CSV from model predictions on the test set.
    
    Args:
        model: Trained Keras model.
        X_test: Test data for predictions.
        model_image_size: Target size for images.
        csv_name: Output CSV file name.
    """
    # 对测试数据进行预测
    y_pred = model.predict(X_test, verbose=1)
    print("Predictions (first 3):", y_pred[:3])  # 打印前3个预测结果
    
    # 裁剪预测值，确保它们在合理范围内
    y_pred = y_pred.clip(min=0.005, max=0.995)
    print("Clipped Predictions (first 3):", y_pred[:3])  # 打印裁剪后的前3个预测结果

    # 创建图像数据生成器
    gen = ImageDataGenerator()

    # 使用 flow_from_directory 加载测试数据集
    test_generator = gen.flow_from_directory(
        dir + "/test/",  # 测试集文件夹路径
        target_size=(model_image_size, model_image_size),  # 调整图像大小
        shuffle=False,  # 不打乱图像顺序
        batch_size=16,  # 每批处理的图像数量
        class_mode=None  # 没有标签
    )

    # 存储文件名和预测值的列表
    result_list = []
    
    # 遍历测试集文件名并与预测结果结合
    for i, fname in enumerate(test_generator.filenames):
        name = os.path.basename(fname)  # 获取文件名（去掉路径部分）
        result_list.append([name, *y_pred[i]])

    # 转换为 NumPy 数组
    result_array = np.array(result_list)

    # 创建字典，存储图片名称和预测概率
    data = {'img': result_array[:, 0]}  # 图片名称
    for i in range(10):
        data[f"c{i}"] = result_array[:, i + 1]  # 每个类别的预测概率

    # 转换为 DataFrame
    df = pd.DataFrame(data, columns=['img'] + [f'c{i}' for i in range(10)])

    # 打印前10行数据检查
    print(df.head(10))

    # 按照图片名称排序
    df = df.sort_values(by='img')

    # 将 DataFrame 保存为 CSV 文件
    df.to_csv(csv_name, index=False, float_format='%.3f')

    # 输出文件数量和预测结果数量的简单检查
    print(f"Number of files: {len(test_generator.filenames)}")
    print(f"Number of predictions: {len(y_pred)}")

# 加载已保存的模型
model_mix = load_model("models/mixed-model.h5")

# 测试数据文件路径
dir = "F:/JupyterWorkSpace/BBBBBBS"

# 预训练模型列表
test_premodels = [
    "bottleneck_ResNet50_test.h5", 
    "bottleneck_Xception_test.h5", 
    "bottleneck_InceptionV3_test.h5",
]

# 得到混合输入
X_test = []
for filename in test_premodels:
    print(f"------------------ {filename}")
    with h5py.File(os.path.join("models", "finetune", filename), 'r') as h:
        X_test.append(np.array(h['test']))
        
# 合并多个模型的测试数据
X_test = np.concatenate(X_test, axis=1)

# 生成Kaggle CSV
gen_kaggle_csv(model_mix, X_test, 320, 'csv/mixed-pred-test.csv')

# 打印完成信息
print("done")


------------------ bottleneck_ResNet50_test.h5
------------------ bottleneck_Xception_test.h5
------------------ bottleneck_InceptionV3_test.h5
2492/2492 [==============================] - 6s 2ms/step
Predictions (first 3): [[0.0063405  0.01604431 0.02785161 0.02301869 0.03530462 0.8023355
  0.02995177 0.01099865 0.01762458 0.03052977]
 [0.02174795 0.01262218 0.02817013 0.01803399 0.02463043 0.81699604
  0.01878481 0.00956798 0.04342525 0.00602125]
 [0.16492495 0.07910203 0.05841956 0.06612758 0.09217478 0.03954029
  0.07231129 0.11545973 0.11070835 0.20123146]]
Clipped Predictions (first 3): [[0.0063405  0.01604431 0.02785161 0.02301869 0.03530462 0.8023355
  0.02995177 0.01099865 0.01762458 0.03052977]
 [0.02174795 0.01262218 0.02817013 0.01803399 0.02463043 0.81699604
  0.01878481 0.00956798 0.04342525 0.00602125]
 [0.16492495 0.07910203 0.05841956 0.06612758 0.09217478 0.03954029
  0.07231129 0.11545973 0.11070835 0.20123146]]
Found 79726 images belonging to 1 classes.
            

# 单个预测

In [1]:
# 指定图片路径
image_path = 'F:\\JupyterWorkSpace\\BBBBBBS\\train\\c1\\img_6.jpg'  # 请确保这条路径是有效的

In [6]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import h5py

def extract_features_from_models(test_premodels, image_path, model_image_size):
    """
    Extract features for a single image from multiple pre-trained models.
    
    Args:
        test_premodels: List of pre-trained model file paths.
        image_path: Path to the image for prediction.
        model_image_size: Target size for image resizing.
    
    Returns:
        X_test: Extracted features from all pre-trained models.
    """
    X_test = []
    
    # 加载并预处理图像
    img = load_img(image_path, target_size=(model_image_size, model_image_size))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 增加批量维度

    # 从每个预训练模型中提取特征
    for model_file in test_premodels:
        with h5py.File(model_file, 'r') as h:
            features = np.array(h['test'])  # 提取预训练模型的特征
            X_test.append(features)

    # 合并所有特征
    X_test = np.concatenate(X_test, axis=1)
    return X_test

def predict_with_mixed_model(model, X_test):
    """
    Predict the class of a single image using the mixed model and extracted features.
    
    Args:
        model: Trained mixed model.
        X_test: Features extracted from pre-trained models.
    """
    # 使用混合模型进行预测
    y_pred = model.predict(X_test)
    print(f"Prediction: {y_pred}")
    
    # 裁剪预测结果，确保在合理范围内
    y_pred = y_pred.clip(min=0.005, max=0.995)
    print(f"Clipped Prediction: {y_pred}")

# 加载混合模型
model_mix = load_model("models/mixed-model.h5")

# 预训练模型文件路径
test_premodels = [
    "models/Single/bottleneck_ResNet50_test.h5",
    "models/Single/bottleneck_Xception_test.h5",
    "models/Single/bottleneck_InceptionV3_test.h5"
]


# 从预训练模型中提取特征
X_test = extract_features_from_models(test_premodels, image_path, 320)

# 使用混合模型进行预测
predict_with_mixed_model(model_mix, X_test)


1/1 [==============================] - 0s 35ms/step
Prediction: [[0.01973563 0.69541645 0.0319957  0.04667007 0.0370991  0.03033314
  0.06915937 0.02763282 0.02671098 0.01524675]]
Clipped Prediction: [[0.01973563 0.69541645 0.0319957  0.04667007 0.0370991  0.03033314
  0.06915937 0.02763282 0.02671098 0.01524675]]


# 上面的部分已经好了，说明错误的地方是bottleneck_ResNet50_test等文件的生成
下面进行修改

In [ ]:
"""
本来的，不要运行！！！！！！！！！！！！！！还是这里有问题，需要改
"""
def write_gap_test(tag, MODEL, weight_file, image_size, lambda_func=None, featurewise_std_normalization=True):
    # 输入张量
    input_tensor = Input((*image_size, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)  # 使用lambda函数进行额外的预处理
    # 加载基本模型，不包括顶部的全连接层
    base_model = MODEL(input_tensor=x, weights=None, include_top=False)
    # 构建最终的模型，使用全局平均池化
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    # 加载权重文件
    model.load_weights("models/" + weight_file, by_name=True)

    print(MODEL.__name__)
    """
    应该是要从这开始改（下面）
    或者，直接把test文件夹里面就放一张数据试试？
    需要注意，新建test0文件夹，
    输出的test.h5不要放一块（改tag，新建tag文件夹）
    """
    # 定义测试数据的生成器
    gen = ImageDataGenerator(
        featurewise_std_normalization=featurewise_std_normalization,
        samplewise_std_normalization=False,
    )
    
    batch_size = 64
    # 加载测试数据
    test_generator = gen.flow_from_directory(
        os.path.join(dir, 'test'),
        target_size=image_size,
        shuffle=False,
        batch_size=batch_size,
        class_mode=None  # 不使用标签
    )
    
    print("predict_generator test {}".format(math.ceil(test_generator.samples // batch_size + 1)))
    
    # 预测测试数据
    test = model.predict(test_generator, steps=math.ceil(test_generator.samples // batch_size + 1))
    print("test: {}".format(test.shape))

    # 将预测结果保存到文件
    print("begin create database {}".format(MODEL.__name__))
    with h5py.File(os.path.join("models", tag, "bottleneck_{}_test.h5".format(MODEL.__name__)),'w') as h:
        h.create_dataset("test", data=test)
    
    print("write_gap_test {} succeeded".format(MODEL.__name__))
    
resnet50_weight_file = "resnet50-imagenet-finetune152.h5"
xception_weight_file = "xception-imagenet-finetune116.h5"
inceptionV3_weight_file = "inceptionV3-imagenet-finetune172.h5"
print("===== Test =====")
write_gap_test("finetune", ResNet50, resnet50_weight_file, (240, 320))
write_gap_test("finetune", Xception, xception_weight_file, (320, 480), xception_preprocess_input)
write_gap_test("finetune", InceptionV3, inceptionV3_weight_file, (320, 480), inception_v3_preprocess_input)

In [3]:
from keras.models import Model, load_model
from keras.layers import Input, Lambda, GlobalAveragePooling2D
from keras.applications import ResNet50, Xception, InceptionV3
from keras.preprocessing import image
import numpy as np
import h5py
import os

from tensorflow.keras.preprocessing import image  # 使用 tensorflow.keras

from keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_v3_preprocess_input
from keras.applications.xception import Xception, preprocess_input as xception_preprocess_input

def load_custom_model(MODEL, weight_file, image_size):
    # 输入张量
    input_tensor = Input((*image_size, 3))
    base_model = MODEL(input_tensor=input_tensor, weights=None, include_top=False)
    
    # 加载权重
    base_model.load_weights("models/" + weight_file, by_name=True)
    
    # 构建最终模型
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    return model

def write_gap_test(tag, MODEL, weight_file, image_size, image_path, lambda_func=None, featurewise_std_normalization=True):
    # 检查图片路径是否有效
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        return
    
    # 加载自定义模型
    model = load_custom_model(MODEL, weight_file, image_size)
    
    print(f"Predicting for image: {image_path}")
    
    # 直接加载和预处理单张图片
    img = image.load_img(image_path, target_size=image_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 增加batch维度
    
    # 使用lambda函数进行预处理
    if lambda_func:
        img_array = lambda_func(img_array)
    elif featurewise_std_normalization:
        img_array = img_array / 255.0  # 假设图片值已归一化
    
    # 进行预测
    test = model.predict(img_array)
    print("Prediction result: {}".format(test.shape))

    # 将预测结果保存到文件
    output_dir = os.path.join("models", tag)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Begin creating database for {MODEL.__name__}")
    with h5py.File(os.path.join(output_dir, f"bottleneck_{MODEL.__name__}_test.h5"), 'w') as h:
        h.create_dataset("test", data=test)
    
    print(f"write_gap_test {MODEL.__name__} succeeded")

# 测试不同模型时指定的图片路径
image_path = 'F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg'  # 确保这条路径是有效的

resnet50_weight_file = "resnet50-imagenet-finetune152.h5"
xception_weight_file = "xception-imagenet-finetune116.h5"
inceptionV3_weight_file = "inceptionV3-imagenet-finetune172.h5"

print("===== Test =====")
write_gap_test("Single", ResNet50, resnet50_weight_file, (240, 320), image_path)
write_gap_test("Single", Xception, xception_weight_file, (320, 480), image_path, xception_preprocess_input)


===== Test =====
Predicting for image: F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg
1/1 [==============================] - 1s 680ms/step
Prediction result: (1, 2048)
Begin creating database for ResNet50
write_gap_test ResNet50 succeeded
Predicting for image: F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg
1/1 [==============================] - 1s 570ms/step
Prediction result: (1, 2048)
Begin creating database for Xception
write_gap_test Xception succeeded


In [5]:
# 需要运行两次
write_gap_test("Single", InceptionV3, inceptionV3_weight_file, (320, 480), image_path, inception_v3_preprocess_input)

Predicting for image: F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg
1/1 [==============================] - 1s 825ms/step
Prediction result: (1, 2048)
Begin creating database for InceptionV3
write_gap_test InceptionV3 succeeded


#  单个预测，汇总

In [14]:
# 指定图片路径
image_path = 'F:\\JupyterWorkSpace\\BBBBBBS\\train\\c1\\img_6.jpg'  # 请确保这条路径是有效的
# image_path = 'F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg'  # 确保这条路径是有效的

from keras.models import Model, load_model
from keras.layers import Input, Lambda, GlobalAveragePooling2D
from keras.applications import ResNet50, Xception, InceptionV3
from keras.preprocessing import image
import numpy as np
import h5py
import os

from tensorflow.keras.preprocessing import image  # 使用 tensorflow.keras

from keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_v3_preprocess_input
from keras.applications.xception import Xception, preprocess_input as xception_preprocess_input

def load_custom_model(MODEL, weight_file, image_size):
    # 输入张量
    input_tensor = Input((*image_size, 3))
    base_model = MODEL(input_tensor=input_tensor, weights=None, include_top=False)
    
    # 加载权重
    base_model.load_weights("models/" + weight_file, by_name=True)
    
    # 构建最终模型
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    return model

def write_gap_test(tag, MODEL, weight_file, image_size, image_path, lambda_func=None, featurewise_std_normalization=True):
    # 检查图片路径是否有效
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        return
    
    # 加载自定义模型
    model = load_custom_model(MODEL, weight_file, image_size)
    
    print(f"Predicting for image: {image_path}")
    
    # 直接加载和预处理单张图片
    img = image.load_img(image_path, target_size=image_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 增加batch维度
    
    # 使用lambda函数进行预处理
    if lambda_func:
        img_array = lambda_func(img_array)
    elif featurewise_std_normalization:
        img_array = img_array / 255.0  # 假设图片值已归一化
    
    # 进行预测
    test = model.predict(img_array)
    print("Prediction result: {}".format(test.shape))

    # 将预测结果保存到文件
    output_dir = os.path.join("models", tag)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Begin creating database for {MODEL.__name__}")
    with h5py.File(os.path.join(output_dir, f"bottleneck_{MODEL.__name__}_test.h5"), 'w') as h:
        h.create_dataset("test", data=test)
    
    print(f"write_gap_test {MODEL.__name__} succeeded")

# 测试不同模型时指定的图片路径
image_path = 'F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg'  # 确保这条路径是有效的

resnet50_weight_file = "resnet50-imagenet-finetune152.h5"
xception_weight_file = "xception-imagenet-finetune116.h5"
inceptionV3_weight_file = "inceptionV3-imagenet-finetune172.h5"

print("===== Test =====")
write_gap_test("Single", ResNet50, resnet50_weight_file, (240, 320), image_path)
write_gap_test("Single", Xception, xception_weight_file, (320, 480), image_path, xception_preprocess_input)
# 需要运行两次
write_gap_test("Single", InceptionV3, inceptionV3_weight_file, (320, 480), image_path, inception_v3_preprocess_input)

import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import h5py

def extract_features_from_models(test_premodels, image_path, model_image_size):
    """
    Extract features for a single image from multiple pre-trained models.
    
    Args:
        test_premodels: List of pre-trained model file paths.
        image_path: Path to the image for prediction.
        model_image_size: Target size for image resizing.
    
    Returns:
        X_test: Extracted features from all pre-trained models.
    """
    X_test = []
    
    # 加载并预处理图像
    img = load_img(image_path, target_size=(model_image_size, model_image_size))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 增加批量维度

    # 从每个预训练模型中提取特征
    for model_file in test_premodels:
        with h5py.File(model_file, 'r') as h:
            features = np.array(h['test'])  # 提取预训练模型的特征
            X_test.append(features)

    # 合并所有特征
    X_test = np.concatenate(X_test, axis=1)
    return X_test

def predict_with_mixed_model(model, X_test):
    """
    Predict the class of a single image using the mixed model and extracted features.
    
    Args:
        model: Trained mixed model.
        X_test: Features extracted from pre-trained models.
    """
    # 使用混合模型进行预测
    y_pred = model.predict(X_test)
    print(f"Prediction: {y_pred}")
    
    # 裁剪预测结果，确保在合理范围内
    y_pred = y_pred.clip(min=0.005, max=0.995)
    print(f"Clipped Prediction: {y_pred}")

# 加载混合模型
model_mix = load_model("models/mixed-model.h5")

# 预训练模型文件路径
test_premodels = [
    "models/Single/bottleneck_ResNet50_test.h5",
    "models/Single/bottleneck_Xception_test.h5",
    "models/Single/bottleneck_InceptionV3_test.h5"
]


# 从预训练模型中提取特征
X_test = extract_features_from_models(test_premodels, image_path, 320)

# 使用混合模型进行预测
predict_with_mixed_model(model_mix, X_test)


===== Test =====
Predicting for image: F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg
1/1 [==============================] - 1s 649ms/step
Prediction result: (1, 2048)
Begin creating database for ResNet50
write_gap_test ResNet50 succeeded
Predicting for image: F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg
1/1 [==============================] - 1s 610ms/step
Prediction result: (1, 2048)
Begin creating database for Xception
write_gap_test Xception succeeded
Predicting for image: F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg
1/1 [==============================] - 1s 849ms/step
Prediction result: (1, 2048)
Begin creating database for InceptionV3
write_gap_test InceptionV3 succeeded
1/1 [==============================] - 0s 32ms/step
Prediction: [[0.10817979 0.13617077 0.12071446 0.14535302 0.12824251 0.07051165
  0.06610069 0.07514649 0.08422078 0.06535983]]
Clipped Prediction: [[0.10817979 0.13617077 0.12071446 0.14535302 0.12824251 0.07051165
  0.06610069 0.07514649 0.08422078 0.

In [19]:
import os
import numpy as np
import h5py
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50, Xception, InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_v3_preprocess_input
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess_input

import tensorflow as tf
import random
np.random.seed(8147)
random.seed(8147)
tf.random.set_seed(8147)


def load_custom_model(model_class, weight_file, image_size, weights_dir="models"):
    """
    Load a custom model with pre-trained weights.
    
    Args:
        model_class: Pre-trained model class (e.g., ResNet50, Xception, etc.).
        weight_file: Filename of the pre-trained weights.
        image_size: Target image size for the model input.
        weights_dir: Directory containing the model weights (default is 'models').
    
    Returns:
        model: The custom model with the pre-trained weights loaded.
    """
    input_tensor = Input((*image_size, 3))
    base_model = model_class(input_tensor=input_tensor, weights=None, include_top=False)
    
    # Load weights
    weight_path = os.path.join(weights_dir, weight_file)
    if not os.path.exists(weight_path):
        raise FileNotFoundError(f"Weight file not found: {weight_path}")
    
    base_model.load_weights(weight_path, by_name=True)
    
    # Build final model
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    return model


def write_gap_test(tag, model_class, weight_file, image_size, image_path, preprocess_input_func=None, featurewise_std_normalization=True, weights_dir="models"):
    """
    Process a single image and extract features using a custom model, then save the features to an HDF5 file.
    
    Args:
        tag: Tag to organize output directory.
        model_class: Pre-trained model class (e.g., ResNet50, Xception, etc.).
        weight_file: Filename of the pre-trained weights.
        image_size: Target image size for the model input.
        image_path: Path to the image to be processed.
        preprocess_input_func: Preprocessing function specific to the model.
        featurewise_std_normalization: Whether to apply featurewise standard normalization (default is True).
        weights_dir: Directory containing the model weights (default is 'models').
    """
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        return
    
    model = load_custom_model(model_class, weight_file, image_size, weights_dir)
    
    print(f"Predicting for image: {image_path}")
    
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=image_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Apply preprocessing if necessary
    if preprocess_input_func:
        img_array = preprocess_input_func(img_array)
    elif featurewise_std_normalization:
        img_array = img_array / 255.0  # Normalize pixel values
    
    # Predict features
    test = model.predict(img_array)
    print("Prediction result: {}".format(test.shape))

    # Save the features to an HDF5 file
    output_dir = os.path.join("models", tag)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Saving features for {model_class.__name__}")
    with h5py.File(os.path.join(output_dir, f"bottleneck_{model_class.__name__}_test.h5"), 'w') as h:
        h.create_dataset("test", data=test)
    
    print(f"write_gap_test for {model_class.__name__} succeeded")


def extract_features_from_models(model_files, image_path, model_image_size):
    """
    Extract features from a single image using multiple pre-trained models.
    
    Args:
        model_files: List of HDF5 files containing extracted features from pre-trained models.
        image_path: Path to the image to be processed.
        model_image_size: Target size for image resizing.
    
    Returns:
        X_test: Combined features extracted from all models.
    """
    X_test = []
    
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(model_image_size, model_image_size))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Extract features from each pre-trained model
    for model_file in model_files:
        with h5py.File(model_file, 'r') as h:
            features = np.array(h['test'])  # Extract features from HDF5 file
            X_test.append(features)

    # Combine all features
    X_test = np.concatenate(X_test, axis=1)
    return X_test


def predict_with_mixed_model(model, X_test):
    """
    Predict the class of a single image using the mixed model and extracted features.
    
    Args:
        model: Trained mixed model.
        X_test: Extracted features from pre-trained models.
    """
    y_pred = model.predict(X_test)
    print(f"Prediction: {y_pred}")
    
    # Clip the prediction results to a valid range
    y_pred = y_pred.clip(min=0.005, max=0.995)
    print(f"Clipped Prediction: {y_pred}")


# Paths and filenames
image_path = 'F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg'
resnet50_weight_file = "resnet50-imagenet-finetune152.h5"
xception_weight_file = "xception-imagenet-finetune116.h5"
inceptionV3_weight_file = "inceptionV3-imagenet-finetune172.h5"

print("===== Test =====")
write_gap_test("Single", ResNet50, resnet50_weight_file, (240, 320), image_path)
write_gap_test("Single", Xception, xception_weight_file, (320, 480), image_path, xception_preprocess_input)
write_gap_test("Single", InceptionV3, inceptionV3_weight_file, (320, 480), image_path, inception_v3_preprocess_input)

# Mixed model prediction
model_mix = load_model("models/mixed-model.h5")

test_premodels = [
    "models/Single/bottleneck_ResNet50_test.h5",
    "models/Single/bottleneck_Xception_test.h5",
    "models/Single/bottleneck_InceptionV3_test.h5"
]

# Extract features and predict using the mixed model
X_test = extract_features_from_models(test_premodels, image_path, 320)
predict_with_mixed_model(model_mix, X_test)


===== Test =====
Predicting for image: F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg
1/1 [==============================] - 1s 715ms/step
Prediction result: (1, 2048)
Saving features for ResNet50
write_gap_test for ResNet50 succeeded
Predicting for image: F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg
1/1 [==============================] - 1s 582ms/step
Prediction result: (1, 2048)
Saving features for Xception
write_gap_test for Xception succeeded
Predicting for image: F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg
1/1 [==============================] - 1s 846ms/step
Prediction result: (1, 2048)
Saving features for InceptionV3
write_gap_test for InceptionV3 succeeded
1/1 [==============================] - 0s 36ms/step
Prediction: [[0.11752235 0.12432297 0.04872757 0.23426199 0.12652394 0.02233721
  0.043757   0.0460817  0.0551057  0.18135951]]
Clipped Prediction: [[0.11752235 0.12432297 0.04872757 0.23426199 0.12652394 0.02233721
  0.043757   0.0460817  0.0551057  0.18135951]]


In [16]:
predict_with_mixed_model(model_mix, X_test)


1/1 [==============================] - 0s 15ms/step
Prediction: [[0.08819199 0.09396999 0.12517974 0.12770745 0.15163337 0.07297713
  0.08025378 0.08779748 0.08683237 0.08545671]]
Clipped Prediction: [[0.08819199 0.09396999 0.12517974 0.12770745 0.15163337 0.07297713
  0.08025378 0.08779748 0.08683237 0.08545671]]
